In [1]:
import pandas as pd
import re
from sklearn.utils import shuffle
import random as rand
import numpy as np
import math

In [2]:
random = 24

In [3]:
data = pd.read_csv('games.csv')
random = 24
data = data[["Metacritic", "Indie", "Presence", "Platform", 
             "Memory", "RatingsBreakdown", "ReleaseDate", "Soundtrack", 
             "OriginalCost", "DiscountedCost", "Controller",
             "Achievements", "Languages", "Storage", "Tags"]]

In [4]:
data.describe(include= 'all')

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
count,3356.000000,30045.000000,3.015600e+04,30123,28316,15044,27024,30045.000000,29504,727,29976.000000,30156.000000,30027,27491,30045
unique,NaN,NaN,NaN,2041,699,4806,4133,NaN,396,121,NaN,NaN,3196,2067,24866
top,NaN,NaN,NaN,PC,2 GB RAM,meh: 1,2018-08-22,NaN,$9.99,$0.49,NaN,NaN,English,1 GB available space,"Adventure, Casual, Hidden Object"
freq,NaN,NaN,NaN,15720,6487,910,238,NaN,4144,260,NaN,NaN,17133,2303,258
mean,72.924911,0.740722,8.822923e+04,NaN,NaN,NaN,NaN,0.004760,NaN,NaN,0.457666,41.047984,NaN,NaN,NaN
std,10.805296,0.438246,2.447597e+05,NaN,NaN,NaN,NaN,0.068826,NaN,NaN,0.498213,316.231316,NaN,NaN,NaN
min,20.000000,0.000000,0.000000e+00,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
25%,67.000000,0.000000,1.890000e+02,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
50%,74.000000,1.000000,3.380000e+02,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN
75%,80.000000,1.000000,4.981500e+03,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,1.000000,21.000000,NaN,NaN,NaN


In [5]:
actions = {k:[] for k in data.columns}
actions['Indie'].append(lambda x: x == 1.0) 
actions['Controller'].append(lambda x: x == 1.0) 
actions['Soundtrack'].append(lambda x: x == 1.0)
actions['OriginalCost'].append(lambda x: x.replace('$', ''))
actions['ReleaseDate'].append(lambda x: x.split('-')[0])

def replace_MB(x):
    x = str(x).lower()
    first = re.search(r"\d", x)
    if (first == None):
        return None
    first = first.start()
    i = first+1
    while(i<len(x) and (x[i].isdigit() or x[i] == '.')):
        i = i+1
    last = i
    while(i<len(x) and x[i:i+2] != "gb" and x[i:i+2] != "mb"):
        i = i+1
    if (i == len(x) or x[i:i+2] == "gb"):
        return(float(x[first:last])*1000)
    return(float(x[first:last]))

actions['Storage'].append(lambda x: replace_MB(x))
actions['Memory'].append(lambda x: replace_MB(x))

for name in actions:
    for action in actions[name]:
        data.loc[:,name] = data[data[name].notna()][name].apply(action)
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,Languages,Storage,Tags
0,83.0,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012,False,Free to Play,NaN,True,179.0,"English, Czech, Danish, Dutch, Finnish, French...",15000.0,"FPS, Shooter, Multiplayer, Competitive, Action..."
1,82.0,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017,False,Free To Play,NaN,True,61.0,"English, French, Italian, German, Japanese, Ko...",105000.0,"Free to Play, Looter Shooter, FPS, Multiplayer..."
2,90.0,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013,False,Free to Play,NaN,True,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...",15000.0,"Free to Play, MOBA, Multiplayer, Strategy, eSp..."
3,71.0,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014,False,19.99,NaN,True,0.0,"English, French, German, Russian",85000.0,"RPG, MMORPG, Open World, Massively Multiplayer..."
4,68.0,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018,False,39.99,NaN,True,308.0,"English, French, Italian, German, Russian",50000.0,"Adventure, Action, Pirates, Open World, Multip..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018,False,0.99,NaN,True,0.0,English,10.0,"Indie, Casual"
30246,NaN,True,13.0,PC,6000.0,NaN,2017,False,0.99,NaN,False,0.0,English,2000.0,"Indie, Puzzle, Pixel Graphics, Mystery, Experi..."
30247,NaN,False,11.0,PC,2048.0,NaN,2018,False,1.99,NaN,False,0.0,English,2048.0,Casual
30248,NaN,True,423.0,PC,500.0,NaN,2017,False,0.99,NaN,True,0.0,English,300.0,"Action, Adventure, Indie, Casual"


In [6]:
platform = pd.Series(data['Platform']).str.get_dummies(', ')
platform.columns
data['Platform-Mobile'] = platform[['Android','iOS']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-PC'] = platform[['Apple II','Atari ST', 'Classic Macintosh', 'Commodore / Amiga',  'Linux', 'PC', 'Web', 'macOS']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Xbox'] = platform[['Xbox', 'Xbox 360', 'Xbox One']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-PlayStation'] = platform[['PS Vita', 'PSP', 'PlayStation', 'PlayStation 2', 'PlayStation 3', 'PlayStation 4']].sum(axis = 1).apply(lambda x: x > 0)  
data['Platform-Nintendo'] = platform[['Game Boy', 'GameCube', 'Nintendo 3DS', 'Nintendo DS', 'Nintendo Switch', 'Wii', 'Wii U']].sum(axis = 1).apply(lambda x: x > 0) 
data['Platform-Sega'] = platform[['Dreamcast','Game Gear','Genesis','SEGA Master System', 'SEGA Saturn']].sum(axis = 1).apply(lambda x: x > 0) 
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,...,Achievements,Languages,Storage,Tags,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega
0,83.0,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012,False,Free to Play,NaN,...,179.0,"English, Czech, Danish, Dutch, Finnish, French...",15000.0,"FPS, Shooter, Multiplayer, Competitive, Action...",False,True,True,True,False,False
1,82.0,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017,False,Free To Play,NaN,...,61.0,"English, French, Italian, German, Japanese, Ko...",105000.0,"Free to Play, Looter Shooter, FPS, Multiplayer...",False,True,True,True,False,False
2,90.0,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013,False,Free to Play,NaN,...,0.0,"English, Bulgarian, Czech, Danish, Dutch, Finn...",15000.0,"Free to Play, MOBA, Multiplayer, Strategy, eSp...",False,True,False,False,False,False
3,71.0,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014,False,19.99,NaN,...,0.0,"English, French, German, Russian",85000.0,"RPG, MMORPG, Open World, Massively Multiplayer...",False,True,False,False,False,False
4,68.0,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018,False,39.99,NaN,...,308.0,"English, French, Italian, German, Russian",50000.0,"Adventure, Action, Pirates, Open World, Multip...",False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018,False,0.99,NaN,...,0.0,English,10.0,"Indie, Casual",False,True,False,False,False,False
30246,NaN,True,13.0,PC,6000.0,NaN,2017,False,0.99,NaN,...,0.0,English,2000.0,"Indie, Puzzle, Pixel Graphics, Mystery, Experi...",False,True,False,False,False,False
30247,NaN,False,11.0,PC,2048.0,NaN,2018,False,1.99,NaN,...,0.0,English,2048.0,Casual,False,True,False,False,False,False
30248,NaN,True,423.0,PC,500.0,NaN,2017,False,0.99,NaN,...,0.0,English,300.0,"Action, Adventure, Indie, Casual",False,True,False,False,False,False


In [7]:
def RatingsBreakdown(x):
    x = x.split(', ')
    result = [np.nan, np.nan, np.nan, np.nan]
    for i in x:
        if i.split(' ')[0] == 'recommended:':
            result[0] = i.split(' ')[1]
        elif i.split(' ')[0] == 'meh:':
            result[1] = i.split(' ')[1]
        elif i.split(' ')[0] == 'exceptional:':
            result[2] = i.split(' ')[1]
        elif i.split(' ')[0] == 'skip:':
            result[3] = i.split(' ')[1]
    return result  
        
df_aux = pd.DataFrame(data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].apply(lambda x: RatingsBreakdown(x)))
df_aux = pd.DataFrame(df_aux.RatingsBreakdown.values.tolist(),index = data[data['RatingsBreakdown'].notna()]['RatingsBreakdown'].index, columns=['RatingsBreakdown-Recommended','RatingsBreakdown-Meh','RatingsBreakdown-Exceptional','RatingsBreakdown-Skip'])
data = data.join(df_aux)
data

,Metacritic,Indie,Presence,Platform,Memory,RatingsBreakdown,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,...,Platform-Mobile,Platform-PC,Platform-Xbox,Platform-PlayStation,Platform-Nintendo,Platform-Sega,RatingsBreakdown-Recommended,RatingsBreakdown-Meh,RatingsBreakdown-Exceptional,RatingsBreakdown-Skip
0,83.0,False,1009588.0,"PC, Xbox 360, PlayStation 3",2000.0,"recommended: 882, meh: 501, exceptional: 283, ...",2012,False,Free to Play,NaN,...,False,True,True,True,False,False,882,501,283,185
1,82.0,False,1007425.0,"PlayStation 5, Web, Xbox Series X, PC, Xbox On...",6000.0,"recommended: 726, meh: 465, exceptional: 227, ...",2017,False,Free To Play,NaN,...,False,True,True,True,False,False,726,465,227,106
2,90.0,False,1009306.0,"Linux, macOS, PC",4000.0,"meh: 496, recommended: 446, skip: 371, excepti...",2013,False,Free to Play,NaN,...,False,True,False,False,False,False,446,496,228,371
3,71.0,False,1000781.0,PC,3000.0,"recommended: 21, meh: 12, exceptional: 5, skip: 1",2014,False,19.99,NaN,...,False,True,False,False,False,False,21,12,5,1
4,68.0,False,777456.0,"PC, Xbox One",4000.0,"recommended: 198, meh: 131, exceptional: 62, s...",2018,False,39.99,NaN,...,False,True,True,False,False,False,198,131,62,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30245,NaN,True,105.0,PC,512.0,NaN,2018,False,0.99,NaN,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30246,NaN,True,13.0,PC,6000.0,NaN,2017,False,0.99,NaN,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30247,NaN,False,11.0,PC,2048.0,NaN,2018,False,1.99,NaN,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN
30248,NaN,True,423.0,PC,500.0,NaN,2017,False,0.99,NaN,...,False,True,False,False,False,False,NaN,NaN,NaN,NaN


In [8]:
Tags = pd.Series(data['Tags']).str.get_dummies(', ').apply(lambda x: x > 0)
del Tags['Indie']
del Tags['Soundtrack']
del Tags['Controller']
Languages = pd.Series(data['Languages']).str.get_dummies(', ').apply(lambda x: x > 0)


data = data.join(Tags)
data = data.join(Languages)

In [9]:
del data['RatingsBreakdown']
del data['Platform']
del data['Tags']
del data['Languages']

In [10]:
rand.seed(random)
#data = data[data['Metacritic'].notna()]
x = [y for y in range(len(data))]
x = rand.sample(x, 2000)
data = data.iloc[x]
data

,Metacritic,Indie,Presence,Memory,ReleaseDate,Soundtrack,OriginalCost,DiscountedCost,Controller,Achievements,...,Polish,Portuguese,Romanian,Russian,Slovakian,Swedish,Thai,Turkish,Ukrainian,Vietnamese
23342,NaN,True,242.0,NaN,2017,False,0.99,NaN,True,0.0,...,False,False,False,False,False,False,False,False,False,False
12545,65.0,True,1000648.0,512.0,2011,False,0.99,NaN,False,0.0,...,False,False,False,False,False,False,False,False,False,False
27518,NaN,False,9.0,4000.0,2020,False,2.50,NaN,False,0.0,...,False,False,False,False,False,False,False,False,False,False
19096,NaN,True,1625.0,1000.0,2014,False,9.99,NaN,True,15.0,...,True,False,False,False,False,False,False,False,False,False
5983,NaN,True,236.0,2000.0,2016,False,9.99,NaN,False,27.0,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24056,NaN,True,218.0,1000.0,2013,False,11.99,NaN,True,0.0,...,False,True,False,False,False,False,False,False,False,False
15334,NaN,False,144.0,NaN,2016,False,NaN,NaN,False,0.0,...,False,False,False,False,False,False,False,False,False,False
8184,NaN,True,127.0,2000.0,NaN,False,4.99,NaN,True,34.0,...,False,False,False,False,False,False,False,False,False,False
24995,NaN,True,128.0,2000.0,2019,False,9.99,NaN,True,0.0,...,False,False,False,False,False,False,False,False,False,False


In [11]:
for name in data.columns:
    print(name, data[name].isna().sum()/2000)

Metacritic 0.8815
Indie 0.006
Presence 0.0035
Memory 0.064
ReleaseDate 0.1095
Soundtrack 0.006
OriginalCost 0.0235
DiscountedCost 0.977
Controller 0.0075
Achievements 0.0035
Storage 0.0865
Platform-Mobile 0.0
Platform-PC 0.0
Platform-Xbox 0.0
Platform-PlayStation 0.0
Platform-Nintendo 0.0
Platform-Sega 0.0
RatingsBreakdown-Recommended 0.6515
RatingsBreakdown-Meh 0.6385
RatingsBreakdown-Exceptional 0.721
RatingsBreakdown-Skip 0.6515
1980s 0.0
1990's 0.0
2.5D 0.0
2D 0.0
2D Fighter 0.0
2D Platformer 0.0
360 Video 0.0
3D 0.0
3D Fighter 0.0
3D Platformer 0.0
3D Vision 0.0
4 Player Local 0.0
4X 0.0
6DOF 0.0
ATV 0.0
Abstract 0.0
Action 0.0
Action RPG 0.0
Action RTS 0.0
Action Roguelike 0.0
Action-Adventure 0.0
Addictive 0.0
Adventure 0.0
Agriculture 0.0
Aliens 0.0
Alternate History 0.0
Ambient 0.0
America 0.0
Animation & Modeling 0.0
Anime 0.0
Arcade 0.0
Archery 0.0
Arena Shooter 0.0
Artificial Intelligence 0.0
Assassin 0.0
Asymmetric VR 0.0
Asynchronous Multiplayer 0.0
Atmospheric 0.0
Audio 

In [12]:
for name in data.columns:
    lis = data[name].unique()
    print('\033[1m'+name+'\033[0;0m')
    print(lis)

2 1.280e+02]
Storage
[5.000e+02 1.000e+02 8.000e+02 2.000e+02       nan 7.890e+02 2.000e+03
 4.000e+03 3.000e+03 1.500e+03 1.000e+03 1.000e+04 7.650e+02 1.200e+02
 1.900e+04 1.500e+01 2.500e+01 1.200e+03 5.120e+02 6.000e+02 2.700e+04
 5.000e+03 7.804e+02 1.500e+02 6.060e+02 2.365e+03 1.200e+04 6.200e+02
 6.269e+02 3.000e+02 5.500e+02 1.280e+02 4.000e+02 7.000e+02 6.000e+03
 2.500e+02 5.000e+01 9.000e+02 7.000e+03 8.000e+03 6.000e+01 1.539e+03
 1.600e+01 6.400e+01 8.500e+01 2.400e+02 5.200e+02 3.500e+02 1.500e+04
 1.024e+03 5.100e+02 3.000e+01 3.000e+04 8.000e+01 2.000e+01 9.900e+02
 4.000e+04 5.000e+04 3.700e+02 1.100e+04 2.000e+04 1.350e+04 4.500e+02
 2.490e+03 9.000e+01 1.140e+02 2.048e+03 2.900e+01 1.170e+03 1.380e+02
 2.500e+03 1.800e+02 1.510e+02 7.000e+01 2.830e+02 8.500e+02 6.500e+02
 1.440e+02 1.000e+01 2.230e+03 1.100e+01 1.400e+04 1.600e+04 3.500e+01
 5.090e+02 4.700e+01 9.210e+02 1.100e+02 2.400e+03 2.560e+02 4.096e+03
 5.460e+00 7.990e+02 5.500e+03 3.500e+03 5.500e+01 3.750